<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# [UIUC Black Hole](https://arxiv.org/abs/1001.4077) Initial data

## Authors: Terrence Pierre Jacques, Zach Etienne, & Ian Ruchlin

### Formatting improvements courtesy Brandon Clark

## This module sets up UIUC Black Hole initial data ([Liu, Etienne, & Shapiro, PRD 80 121503, 2009](https://arxiv.org/abs/1001.4077)). 

### We can convert from spherical to any coordinate system defined in [reference_metric.py](../edit/reference_metric.py) (e.g., SinhSpherical, Cylindrical, Cartesian, etc.) using the [Exact ADM Spherical-or-Cartesian-to-BSSNCurvilinear converter module](Tutorial-ADM_Initial_Data-Converting_Exact_ADM_Spherical_or_Cartesian_to_BSSNCurvilinear.ipynb)

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This module has been validated to exhibit convergence to zero of the Hamiltonian and momentum constraint violation at the expected order to the exact solution (see plots at bottom of [the exact initial data validation start-to-finish tutorial notebook](Tutorial-Start_to_Finish-BSSNCurvilinear-Setting_up_Exact_Initial_Data.ipynb); momentum constraint violation in non-$\phi$ directions is zero), and all quantities have been validated against the [original SENR code](https://bitbucket.org/zach_etienne/nrpy).

### NRPy+ Source Code for this module: [BSSN/UIUCBlackHole.py](../edit/BSSN/UIUCBlackHole.py)

## Introduction:
UIUC black holes have the advantage of finite coordinate radius in the maximal spin limit. It is therefore excellent for studying very highly spinning black holes. This module sets the UIUC black hole at the origin. 

<a id='toc'></a>

# Table of Contents:  
$$\label{toc}$$

1. [Step 1](#initializenrpy): Set up the needed NRPy+ infrastructure and declare core gridfunctions
1. [Step 2](#bl_radius): The Boyer-Lindquist Radius
    1. [Step 2.a](#define_inner_outer_radii): Define the inner and outer radii
    1. [Step 2.b](#define_bl_radius): Define the Boyer-Lindquist radius
1. [Step 3](#line_element): Define the line element, and extract components of $\gamma_{ij}$
1. [Step 4](#extrinsic_curvature): Define and construct nonzero components of the extrinsic curvature $K_{ij}$
1. [Step 5](#lapse_shift): Construct Lapse function $\alpha$ and components of shift vector $\beta$
1. [Step 6](#code_validation): Code Validation against `BSSN.UIUCBlackHole` NRPy+ module
1. [Step 7](#latex_pdf_output) Output this notebook to $\LaTeX$-formatted PDF file



<a id='initializenrpy'></a>

# Step 1: Set up the needed NRPy+ infrastructure and declare core gridfunctions \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

First, we will import the core modules of Python/NRPy+ and specify the main gridfunctions that we will need.
Second, we set some basic NRPy+ parameters. E.g., set the spatial dimension parameter to 3.

**Inputs for initial data**:

* The black hole mass $M$.
* The dimensionless spin parameter $\chi = a/M$

**Additional variables needed for spacetime evolution**:

* Desired coordinate system Boyer-Lindquist coordinates $(r_{BL}, \theta, \phi)$
<br>
* Desired initial lapse $\alpha$ and shift $\beta^i$. We will choose our gauge conditions as $\alpha=1$ and $\beta^i=B^i=0$. $\alpha = \psi^{-2}$ will yield much better behavior, but the conformal factor $\psi$ depends on the desired *destination* coordinate system (which may not be spherical coordinates).

In [1]:
# Step P0: Load needed modules
import sympy as sp             # SymPy: The Python computer algebra package upon which NRPy+ depends
import NRPy_param_funcs as par # NRPy+: Parameter interface
import indexedexp as ixp       # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support

# All gridfunctions will be written in terms of spherical coordinates (r, th, ph):
r,th,ph = sp.symbols('r th ph', real=True)

thismodule = "UIUCBlackHole"

# Step 0: Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Step 1: Set psi, the conformal factor:

# The UIUC initial data represent a Kerr black hole with mass M
#  and dimensionless spin chi in UIUC quasi-isotropic coordinates,
#   see https://arxiv.org/abs/1001.4077
# Input parameters:
M,chi = par.Cparameters("REAL", thismodule, ["M","chi"],[1.0,0.99])

# Spin per unit mass
a = M*chi

<a id='bl_radius'></a>

# Step 2: The Boyer-Lindquist Radius \[Back to [top](#toc)\]
$$\label{bl_radius}$$


<a id='define_inner_outer_radii'></a>

## Step 2.a: Defining the Inner and Outer Radii \[Back to [top](#toc)\]
$$\label{define_inner_outer_radii}$$



Boyer-Lindquist radii of the outer (+) and inner (−) horizons of the BH, defined under equation 1 in [Liu, Etienne, & Shapiro (2009)](https://arxiv.org/abs/1001.4077) as 
$$ r_{\pm} = M \pm \sqrt{M^2 - a^2}$$

In [2]:
# Defined under equation 1 in Liu, Etienne, & Shapiro (2009)
# https://arxiv.org/pdf/1001.4077.pdf

# Boyer - Lindquist outer horizon
rp = M + sp.sqrt(M**2 - a**2)
# Boyer - Lindquist inner horizon
rm = M - sp.sqrt(M**2 - a**2)

<a id='define_bl_radius'></a>

## Step 2.b: Define the Boyer-Lindquist Radius  \[Back to [top](#toc)\]
$$\label{define_bl_radius}$$

Define $r_{BL}$, equation 11 of [Liu, Etienne, & Shapiro (2009)](https://arxiv.org/abs/1001.4077), using the radial coordinate $r$:

$$  r_{BL} = r \left( 1 + \frac{r_+}{4r}\right)^2.  $$

In [3]:
# Boyer - Lindquist radius in terms of UIUC radius
# Eq. 11
# r_{BL} = r * ( 1 + r_+ / 4r )^2
rBL = r*(1 + rp / (4*r))**2

Quantities used to calculate the spatial metric $\gamma_{ij}$, found under equation 2 of [Liu, Etienne, & Shapiro (2009)](https://arxiv.org/abs/1001.4077):
$$  \Sigma = r_{BL}^2 + a^2 \cos^2 \theta, $$

$$ \Delta = r_{BL}^2 - 2Mr_{BL} + a^2,  $$ 

$$  A = \left(r_{BL}^2 + a^2\right)^2 - \Delta a^2 \sin^2 \theta.  $$ 

In [4]:
# Expressions found below Eq. 2
# Sigma = r_{BL}^2 + a^2 cos^2 theta
SIG = rBL**2 + a**2*sp.cos(th)**2

# Delta = r_{BL}^2 - 2Mr_{BL} + a^2
DEL = rBL**2 - 2*M*rBL + a**2

# A = (r_{BL}^2 + a^2)^2 - Delta a^2 sin^2 theta
AA = (rBL**2 + a**2)**2 - DEL*a**2*sp.sin(th)**2

<a id='line_element'></a>

# Step 3: Define the Line element and extract components of $\gamma_{ij}$ \[Back to [top](#toc)\]
$$\label{line_element}$$

The line element, defined in equation 13 of [Liu, Etienne, & Shapiro (2009)](https://arxiv.org/abs/1001.4077):

$$ ds^2 = \frac{\Sigma\left(r + \frac{r_+}{4}\right)^2 } {r^3 \left(r_{BL} - r_- \right)} dr^2 + \Sigma d\theta^2  +  \frac{ A \sin^2 \theta  }{\Sigma} d\phi^2  $$

In [5]:
# *** The ADM 3-metric in spherical basis ***
gammaSphDD = ixp.zerorank2()
# Declare the nonzero components of the 3-metric (Eq. 13):

# ds^2 = Sigma (r + r_+/4)^2 / ( r^3 (r_{BL} - r_- ) * dr^2 +
# Sigma d theta^2  +  (A sin^2 theta) / Sigma  *  d\phi^2

gammaSphDD[0][0] = ((SIG*(r + rp/4)**2)/(r**3*(rBL - rm)))
gammaSphDD[1][1] = SIG
gammaSphDD[2][2] = AA/SIG*sp.sin(th)**2

<a id='extrinsic_curvature'></a>

# Step 4: Define and construct nonzero components of extrinsic curvature $K_{ij}$ \[Back to [top](#toc)\]
$$\label{extrinsic_curvature}$$



Nonzero components of the extrinsic curvature, equation 14 of [Liu, Etienne, & Shapiro (2009)](https://arxiv.org/abs/1001.4077):

$$ K_{r\phi} = K_{\phi r} = \frac{Ma\sin^2\theta}{\Sigma\sqrt{A\Sigma}} \ 
    \left[3r^4_{BL} + 2a^2 r^2_{BL} - a^4 - a^2 \left(r^2_{BL} - a^2\right) \sin^2 \theta\right] \
    \left(1 + \frac{r_+}{4r}\right) \frac{1}{\sqrt{r(r_{BL} - r_-)}}  $$

In [6]:
# *** The physical trace-free extrinsic curvature in spherical basis ***
# Nonzero components of the extrinsic curvature K, given by
# Eq. 14 of Liu, Etienne, & Shapiro, https://arxiv.org/pdf/1001.4077.pdf:
KSphDD     = ixp.zerorank2() # K_{ij} = 0 for these initial data


# K_{r phi} = K_{phi r} = (Ma sin^2 theta) / (Sigma sqrt{A Sigma}) *
#     [3r^4_{BL} + 2a^2 r^2_{BL} - a^4 - a^2 (r^2_{BL} - a^2) sin^2 theta] *
#     (1 + r_+ / 4r) (1 / sqrt{r(r_{BL} - r_-)})

KSphDD[0][2] = KSphDD[2][0] = (M*a*sp.sin(th)**2)/(SIG*sp.sqrt(AA*SIG))*\
                (3*rBL**4 + 2*a**2*rBL**2 - a**4- a**2*(rBL**2 - a**2)*\
                 sp.sin(th)**2)*(1 + rp/(4*r))*1/sp.sqrt(r*(rBL - rm))

Nonzero components of the extrinsic curvature, equation 15 of [Liu, Etienne, & Shapiro (2009)](https://arxiv.org/abs/1001.4077):

$$ K_{\theta\phi} = K_{\phi\theta} = -\frac{2a^3 Mr_{BL}\cos\theta \sin^3\theta} {\Sigma \sqrt{A\Sigma} } \left(r - \frac{r_+}{4}\right) \sqrt {\frac{r_{BL} - r_-}{r} }  $$

In [7]:
# Components of the extrinsic curvature K, given by
# Eq. 15 of Liu, Etienne, & Shapiro, https://arxiv.org/pdf/1001.4077.pdf:

# K_{theta phi} = K_{phi theta} = -(2a^3 Mr_{BL} cos theta sin^3 theta) /
#         (Sigma sqrt{A Sigma}) x (r - r_+ / 4) sqrt{(r_{BL} - r_-) / r }

KSphDD[1][2] = KSphDD[2][1] = -((2*a**3*M*rBL*sp.cos(th)*sp.sin(th)**3)/ \
                (SIG*sp.sqrt(AA*SIG)))*(r - rp/4)*sp.sqrt((rBL - rm)/r)

<a id='lapse_shift'></a>

# Step 5: Construct Lapse function $\alpha$ and components of shift vector $\beta$ \[Back to [top](#toc)\]
$$\label{lapse_shift}$$

$$\alpha=1$$ 
<br>
$$\beta^i=B^i=0$$

In [8]:
alphaSph = sp.sympify(1)
betaSphU = ixp.zerorank1() # We generally choose \beta^i = 0 for these initial data
BSphU    = ixp.zerorank1() # We generally choose B^i = 0 for these initial data

# Validated against original SENR: KSphDD[0][2], KSphDD[1][2], gammaSphDD[2][2], gammaSphDD[0][0], gammaSphDD[1][1]
# print(sp.mathematica_code(gammaSphDD[1][1]))

<a id='code_validation'></a>

# Step 6: Code Validation against `BSSN.UIUCBlackHole` NRPy+ module \[Back to [top](#toc)\]
$$\label{code_validation}$$


Here, as a code validation check, we verify agreement in the SymPy expressions for UIUC black hole initial data between

1. this tutorial and 
2. the NRPy+ [BSSN.UIUCBlackHole](../edit/BSSN/UIUCBlackHole.py) module.

In [9]:
# First we import reference_metric, which is
#   needed since BSSN.UIUCBlackHole calls
#   BSSN.ADM_Exact_Spherical_or_Cartesian_to_BSSNCurvilinear, which
#   depends on reference_metric:
import reference_metric as rfm   # NRPy+: Reference metric support
rfm.reference_metric()

import BSSN.UIUCBlackHole as uibh
uibh.UIUCBlackHole()

print("Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module. ALL SHOULD BE ZERO.")
print("alphaSph - uibh.alphaSph = "+str(sp.simplify(alphaSph - uibh.alphaSph)))
for i in range(DIM):
    print("betaSphU["+str(i)+"] - uibh.betaSphU["+str(i)+"] = "+\
          str(sp.simplify(betaSphU[i] - uibh.betaSphU[i])))
    print("BSphU["+str(i)+"] - uibh.BaSphU["+str(i)+"] = "+str(sp.simplify(BSphU[i] - uibh.BSphU[i])))
    for j in range(DIM):
        print("gammaSphDD["+str(i)+"]["+str(j)+"] - uibh.gammaSphDD["+str(i)+"]["+str(j)+"] = "+\
              str(sp.simplify(gammaSphDD[i][j] - uibh.gammaSphDD[i][j])))
        print("KSphDD["+str(i)+"]["+str(j)+"] - uibh.KSphDD["+str(i)+"]["+str(j)+"] = "+\
              str(sp.simplify(KSphDD[i][j] - uibh.KSphDD[i][j])))

Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module. ALL SHOULD BE ZERO.
alphaSph - uibh.alphaSph = 0
betaSphU[0] - uibh.betaSphU[0] = 0
BSphU[0] - uibh.BaSphU[0] = 0
gammaSphDD[0][0] - uibh.gammaSphDD[0][0] = 0
KSphDD[0][0] - uibh.KSphDD[0][0] = 0
gammaSphDD[0][1] - uibh.gammaSphDD[0][1] = 0
KSphDD[0][1] - uibh.KSphDD[0][1] = 0
gammaSphDD[0][2] - uibh.gammaSphDD[0][2] = 0
KSphDD[0][2] - uibh.KSphDD[0][2] = 0
betaSphU[1] - uibh.betaSphU[1] = 0
BSphU[1] - uibh.BaSphU[1] = 0
gammaSphDD[1][0] - uibh.gammaSphDD[1][0] = 0
KSphDD[1][0] - uibh.KSphDD[1][0] = 0
gammaSphDD[1][1] - uibh.gammaSphDD[1][1] = 0
KSphDD[1][1] - uibh.KSphDD[1][1] = 0
gammaSphDD[1][2] - uibh.gammaSphDD[1][2] = 0
KSphDD[1][2] - uibh.KSphDD[1][2] = 0
betaSphU[2] - uibh.betaSphU[2] = 0
BSphU[2] - uibh.BaSphU[2] = 0
gammaSphDD[2][0] - uibh.gammaSphDD[2][0] = 0
KSphDD[2][0] - uibh.KSphDD[2][0] = 0
gammaSphDD[2][1] - uibh.gammaSphDD[2][1] = 0
KSphDD[2][1] - uibh.KSphDD[2][1] = 0
gammaSphDD[

<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-ADM_Initial_Data-UIUC_BlackHole.pdf](Tutorial-ADM_Initial_Data-UIUC_BlackHole.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [10]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-ADM_Initial_Data-UIUC_BlackHole")

Created Tutorial-ADM_Initial_Data-UIUC_BlackHole.tex, and compiled LaTeX
    file to PDF file Tutorial-ADM_Initial_Data-UIUC_BlackHole.pdf
